In [ ]:
# Imports
import sys
import gurobipy as gp
import numpy as np

In [ ]:
# Parameters (Valus of Instance 1)
NUM_VAGOES = 2
NUM_TRECHOS = 6
NUM_ATIVIDADES = 15
NUM_TRENS = 2
P = 5 # tempo de circulação de um vagão j em um trecho q
M = 2880
VAGOES_TREM_SAIDA = [1,1] #[2,2]
VAGOES_TREM_CHEGADA = [0,0] #[1,1]

In [ ]:
# Generate AMV
# AMV[q,s] = 1 se o trecho q é ligado ao trecho s (q vizinho de s ou existe AMV entre q e s); 0 caso contrário
AMV = [[1,1,0,0,0,0],
       [1,1,1,0,1,0],
       [0,1,1,1,0,0],
       [0,0,1,1,0,0],
       [0,1,0,0,1,1],
       [0,0,0,0,1,1]]

In [ ]:
# Generate PSA
# PSA[j] = trecho do pátio onde o vagão j deverá estar posicionado para que o trem t ∈ Tβ |j ∈ t possa sair
PSA = [1,2]

In [ ]:
# Generate PCH
# PCH[j] = trecho do pátio onde o vagão j estará posicionado ao receber o trem t ∈ Tα|j ∈ t
PCH = [2,1]

In [ ]:
# Generate Release
# RELEASE[j] = instante mais cedo que o vagão j está disponível para entrar no pátio (release date), ou seja, horário de chegada do trem t ∈ Tα|j ∈ t
RELEASE = [10,10]

In [ ]:
# Simplify instances informed values
n = NUM_VAGOES
m = NUM_TRECHOS
o = NUM_ATIVIDADES
v = NUM_TRENS
t_beta = VAGOES_TREM_SAIDA
t_alfa = VAGOES_TREM_CHEGADA

In [ ]:
# Initialize model
model = gp.Model("milp")

In [ ]:
# Declare decision variables
var_y = model.addVars(o,n, vtype=gp.GRB.CONTINUOUS, name="y")         ## y[i,j] = instante de início da i-ésima operação do vagão j
var_z = model.addVars(o,m,n, vtype=gp.GRB.BINARY, name="z")           ## z[i,q,j] = 1 se a i-ésima operação do vagão j acontece no trecho q; 0 caso contrário
var_x = model.addVars(m,o,n,o,n, vtype=gp.GRB.BINARY, name="x")       ## x[q,i,j,k,l] = 1 se a i-ésima operação do vagão j precede a k-ésima operação do vagão l no trecho q; 0 caso contrário
var_f = model.addVars(o,n, vtype=gp.GRB.BINARY, name="f")             ## f[i,j] = 1 se a i-ésima operação do vagão j é a última, antes da partida do trem t ∈ Tβ |j ∈ t; 0 caso contrário
var_c = model.addVars(n, vtype=gp.GRB.CONTINUOUS, name="c")           ## C[j] = instante de conclusão do vagão j, horário de partida do trem t ∈ Tβ|j ∈ t
var_c_max = model.addVar(vtype=gp.GRB.CONTINUOUS, name="c_max")       ## Cmax = horário da partida do último vagão j ∈ t, t ∈ Tβ (makespan)

In [ ]:
# Define objective function
model.setObjective(var_c_max, gp.GRB.MINIMIZE)

In [ ]:
# Equation 2
for j in range(n):
  for i in range(o):
    if (i+1 < o):
      model.addConstr
      (
        var_y[i+1,j] 
        >= var_y[i,j] 
        + (P * gp.quicksum(var_z[i,q,j] for q in range(m)))
      )

In [ ]:
# Equation 3
for j in range(n):
  for l in range(n):
    for i in range(o):
      for k in range(o):
        for q in range(m):
          if (i+1 < o):
            model.addConstr
            (
              var_y[i+1,j] 
              >= var_y[k,l]
              - (M * var_x[q,i,j,k,l])
              - (M * (1 - var_z[i,q,j]))
              - (M * (1 - var_z[k,q,l]))
            )

In [ ]:
# Equation 4
for j in range(n):
  for l in range(n):
    for i in range(o):
      for k in range(o):
        for q in range(m):
          if (i+1 < o):
            model.addConstr
            (
              var_y[k,l] 
              >= var_y[i+1,j]
              - (M * (1 - var_x[q,i,j,k,l]))
              - (M * (1 - var_z[i,q,j]))
              - (M * (1 - var_z[k,q,l]))
            )

In [ ]:
# Equation 5
for j in range(n):
  for i in range(o):
    if (i == 0):
      model.addConstr
      (
        gp.quicksum(var_z[i,q,j] for q in range(m))
        <= 1
      )

In [ ]:
# Equation 6
for j in range(n):
  for l in range(n):
    for i in range(o): 
      for k in range(o):
        for q in range(m):
          if (k-1 > 0 and i+1 < o and j != l):
            model.addConstr
            (
              var_x[q,i,j,k,l] 
              + gp.quicksum(var_x[s,k-1,l,i+1,j] for s in range(m))
              <= 1
            )

In [ ]:
# Equation 7
for j in range(n):
  for l in range(n):
    for i in range(o): 
      for k in range(o):
        for q in range(m):
          if (j != l):
            model.addConstr
            (
              var_x[q,i,j,k,l] 
              + var_x[q,k,l,i,j] 
              >= 1
              - (M * (1 - var_z[i,q,j]))
              - (M * (1 - var_z[k,q,l]))
            )

In [ ]:
# Equation 8
for j in range(n):
  for i in range(o):
    if (i-1 > 0):
      model.addConstr
      (
        gp.quicksum(var_z[i,q,j] for q in range(m))
        == 1
        - gp.quicksum(var_f[k,j] for k in range(i-1))
      )

In [ ]:
# Equation 9
for j in t_beta:
  for i in range(o):
    model.addConstr(var_f[i,j] <= var_z[i,PSA[j],j])

In [ ]:
# Equation 10
for j in t_beta:
  model.addConstr(gp.quicksum(var_f[i,j] for i in range(o)) == 1)

In [ ]:
# Equation 11
for j in t_alfa:
  model.addConstr(gp.quicksum(var_f[i,j] for i in range(o)) == 0)

In [ ]:
# Equation 12
for j in range(n):
  for i in range(o):
    for q in range(m):
      if (i == 0 and q == PCH[j]):
        model.addConstr(var_z[i,q,j] == 1)

In [ ]:
# Equation 13
for j in t_beta:
  for i in range(o):
    if (i == 0):
      model.addConstr(var_y[i,j] == RELEASE[j])

In [ ]:
# Equation 14
for j in t_beta:
  for i in range(o):
    if (i == 0):
      model.addConstr(var_y[i,j] >= RELEASE[j])

In [ ]:
# Equation 15
for j in t_alfa:
  for l in t_alfa:
    for i in range(o):
      if (i == 0):
        model.addConstr(var_y[i,j] == var_y[i,l])

In [ ]:
# Equation 16
for j in range(n):
  for i in range(o):
    for q in range(m):
      if (i+1 < o):
        model.addConstr(var_z[i+1,q,j] <= gp.quicksum(AMV[s][q] * var_z[i,s,j] for s in range(m)))

In [ ]:
# Equation 17
for j in t_beta:
  for l in t_beta:
    for i in range(o):
      model.addConstr(var_c[j] >= var_y[i,l])

In [ ]:
# Equation 18
for j in t_alfa:
  for i in range(o):
    if (i == 0):
      model.addConstr(var_c[j] >= var_y[i,j])

In [ ]:
# Equation 19
for j in range(n):
  for i in range(o):
    if (i+1 < o):
      model.addConstr
      (
        var_y[i+1,j]
        >= var_c[j] 
        - M 
        * (1 - gp.quicksum(var_f[k,j] for k in range(i)))
      )

In [ ]:
# Equation 20
for j in t_beta:
  model.addConstr(var_c_max >= var_c[j])

In [ ]:
# Execute model
model.optimize()

In [ ]:
# Print the values of all variables
for v in model.getVars():
    print(f"{v.VarName} = {v.X}")